In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
!pip install spektral

In [64]:

#general libraries
import tensorflow as tf
import numpy as np
import os 
import glob 
import networkx as nx
import pdb
import networkx.algorithms.isomorphism as iso
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

#spektral
from spektral.data import Graph
from spektral.data import Dataset
from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.models import GeneralGNN
from spektral.data import DisjointLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GeneralConv, GatedGraphConv

#tensorflow impors 
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import OGB
from spektral.layers import ECCConv, GlobalSumPool

In [21]:

n1, n2 = 10, 20 # 10 nodes
m1, m2 = 20, 100  # 20 edges
f1 = 100
f2 = 100
seed = 20160  # seed random number generators for reproducibility


In [60]:
from spektral.data import Graph


class MyDataset(Dataset):
    """
    A dataset of random colored graphs.
    The task is to classify each graph with the color which occurs the most in
    its nodes.
    The graphs have `n_colors` colors, of at least `n_min` and at most `n_max`
    nodes connected with probability `p`.
    """

    def __init__(self, n, m, number, **kwargs):
        self.n = n
        self.m = m
        self.number = number
        self.seed = seed 
        super().__init__(**kwargs)

    def read(self):
        def make_graph(i):
            G = nx.gnm_random_graph(self.n[i%2], self.m[i%2], seed=seed)
            return Graph(x=nx.to_numpy_array(G), a=csr_matrix(nx.to_numpy_array(G)), y=i)

        # We must return a list of Graph objects
        print(self.number)
        return [make_graph(_) for _ in range(2*self.number)]


In [62]:
data_train = MyDataset(n=[10,20], m=[20,100], number=100)
data_test = MyDataset(n=[10,20], m=[20,100], number=200)
data_val = MyDataset(n=[10,20], m=[20,100], number=10)

100
200
10


In [63]:
val_loss_array = []
training_loss = []

In [65]:

################################################################################
# Config
################################################################################
learning_rate = 1e-3
epochs = 250
batch_size = 32

################################################################################
# Load data
################################################################################
dataset = data_train  # Set amount=None to train on whole dataset

# Parameters
F = dataset.n_node_features  # Dimension of node features
#S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = data_train+data_val, data_test
# dataset_val = data_val

loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_val = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=1)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=100)

################################################################################
# Build model
dp_rate = 0.7
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv0 = GeneralConv(channels = 64, n_layers = 2, activation="relu", dropout=dp_rate)
        self.conv1 = GeneralConv(channels = 128, n_layers = 2, activation="relu", dropout=dp_rate)
        # self.conv2 = GeneralConv(128, activation="relu", dropout=dp_rate)
        # self.conv3 = GeneralConv(64, activation="relu", dropout=dp_rate)
        self.conv4 = GeneralConv(channels = 64, n_layers = 2,activation="relu", dropout=dp_rate)
        self.global_pool = GlobalSumPool()
        self.dense = Dense(1)

    def call(self, inputs):
        x, a, e = inputs
        x = self.conv0([x, a])
        x = self.conv1([x, a])
        # x = self.conv2([x, a])
        # x = self.conv3([x, a])
        x = self.conv4([x, a])
        output = self.global_pool(x)
        output = self.dense(output)

        return output


model = Net()
optimizer = Adam(learning_rate)
loss_fn1 = BinaryCrossentropy()
loss_fn2 = BinaryCrossentropy()


################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn1(target, predictions) + sum(model.losses)
    # loss_val = 0
    # for batch_val in loader_val:
    #     inputs_val, target_val = batch_val
    #     predictions_val = model(inputs_val, training=False)
    #     loss_val += loss_fn(target_val, predictions_val)
    # loss_val /= loader_val.steps_per_epoch
    # val_loss_array.append(loss_val)
    # print("Validation Loss is: {}".format(loss_val))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        training_loss.append(format(loss / loader_tr.steps_per_epoch))
        loss = 0

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss = 0
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    loss += loss_fn2(target, predictions)
loss /= 100*loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


ValueError: ignored

In [66]:
print(loader_tr)